In [1]:
import pickle
from datetime import datetime

import numpy as np
import pandas as pd
import scipy as sp

# import matplotlib.pyplot as plt

from sklearn import model_selection
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import model_selection
from sklearn.metrics import mean_squared_error, mean_absolute_error

/home/zhanglun/.pyenv/versions/anaconda3-4.1.0/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [43]:
train_features = pd.read_pickle('../dataset/all_feature.pkl').reset_index(drop=True)
train_features.shape

(1080976, 212)

In [3]:
luck = 666

In [4]:
parameters = {
#     'gamma': [0.05],
    #     'n_estimators': [800],
    'max_depth': [2],
#     'learning_rate': [0.15, 0.3],
#         'subsample': [0.9],
#         'colsample_bytree': [0.9],
#     'min_child_weight': [1, 2],
    #     'reg_alpha': [0, 1, 5],
    #     'reg_lambda': [0, 1, 4],
}

In [5]:
model_path = '../dataset/xgbmodel.pkl'

In [6]:
model = XGBClassifier(nthread=4, silent=False, objective= 'binary:logistic', n_estimators=100, seed=luck)

In [7]:
clf = model_selection.GridSearchCV(model, parameters, n_jobs=-1, cv=2, verbose=True)

In [8]:
use_columns = [
    x for x in train_features.columns
    if x not in [
        'orderid', 'uid', 'hotelid', 'basicroomid', 'hotel_roomid', 'roomid',
        'orderlabel'
    ]
]

In [9]:
train_features.shape

(1080976, 212)

In [10]:
clf_feture = train_features.sample(50000, random_state=luck)

In [11]:
%time clf.fit(clf_feture[use_columns], clf_feture['orderlabel'])

Fitting 2 folds for each of 1 candidates, totalling 2 fits


[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:   17.1s finished


CPU times: user 55.6 s, sys: 200 ms, total: 55.8 s
Wall time: 31.9 s


GridSearchCV(cv=2, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=4,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=666, silent=False, subsample=1),
       fit_params={}, iid=True, n_jobs=-1, param_grid={'max_depth': [2]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=True)

In [12]:
clf.best_score_

0.97299999999999998

In [82]:
# # # clf.best_params_['n_estimators'] = 
clf.best_params_['max_depth'] = 4
# # clf.best_params_['gamma'] = 2
# clf.best_params_['subsample'] = 0.8
# clf.best_params_['colsample_bytree'] = 0.8
# # clf.best_params_['n_estimators'] = 1000
clf.best_params_['learning_rate'] = 0.1
# # clf.best_params_['min_child_weight'] = 2
# # clf.best_params_['reg_alpha'] = 0.1

clf.best_params_['scale_pos_weight'] = 5





In [46]:
X_train, X_test, y_train, y_test = train_test_split(
    train_features[use_columns],
    train_features['orderlabel'],
    test_size=0.33,
    random_state=42)

In [83]:
model = XGBClassifier(
    nthread=4,
    silent=False,
    objective='binary:logistic',
    seed=luck,
#     learning_rate=0.3,
    n_estimators=1000,
    **clf.best_params_)
model
# model 

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=4,
       min_child_weight=1, missing=None, n_estimators=1000, nthread=4,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=5, seed=666, silent=False, subsample=1)

In [84]:
model.fit(
    X_train,
    y_train,
    eval_set=[(X_train, y_train), (X_test, y_test)],
    early_stopping_rounds=5,
    eval_metric='auc',
    verbose=True, )

[0]	validation_0-auc:0.872517	validation_1-auc:0.871325
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 5 rounds.
[1]	validation_0-auc:0.877685	validation_1-auc:0.875815
[2]	validation_0-auc:0.886814	validation_1-auc:0.884203
[3]	validation_0-auc:0.888886	validation_1-auc:0.886668
[4]	validation_0-auc:0.889375	validation_1-auc:0.888079
[5]	validation_0-auc:0.890752	validation_1-auc:0.889417
[6]	validation_0-auc:0.893289	validation_1-auc:0.892016
[7]	validation_0-auc:0.894705	validation_1-auc:0.893354
[8]	validation_0-auc:0.895595	validation_1-auc:0.894174
[9]	validation_0-auc:0.89516	validation_1-auc:0.893798
[10]	validation_0-auc:0.896659	validation_1-auc:0.894969
[11]	validation_0-auc:0.897469	validation_1-auc:0.895462
[12]	validation_0-auc:0.898082	validation_1-auc:0.896074
[13]	validation_0-auc:0.897995	validation_1-auc:0.896159
[14]	validation_0-auc:0.898433	validation_1-auc:0.896653


XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=4,
       min_child_weight=1, missing=None, n_estimators=1000, nthread=4,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=5, seed=666, silent=False, subsample=1)

In [70]:
def get_score(model, X_train, train_features):
    result = model.predict_proba(X_train)[:, 1]
    train_features['prob'] = np.nan
    train_features.loc[X_train.index, 'prob'] = result
    train_x = train_features.loc[X_train.index, ['orderid', 'prob', 'orderlabel']]
    train_x.sort_values('prob', ascending=False, inplace=True)
    finall_result_x = train_x.drop_duplicates(['orderid'])
    train_score = finall_result_x.orderlabel.mean()
    print(train_score)
    return train_score

In [85]:
get_score(model, X_train, train_features)

0.323835143303


0.32383514330311181

In [86]:
get_score(model, X_test, train_features)

0.199036972187


0.19903697218667923

In [19]:
model.best_ntree_limit

13

In [87]:
importance_df = pd.DataFrame(model.feature_importances_, index=use_columns)

importance_df.sort_values(0, ascending=False, inplace=True)
importance_df.to_pickle('xgb_importance.pkl')
importance_top50 = ', '.join(['{}:{} '.format(x, importance_df.loc[x].values[0]) for x in importance_df.index][:50])
importance_top50

'hotel_roomid_rank:0.0746765285730362 , basicroomid__room_30days_ordnumratio_max:0.04842883720993996 , roomservice_8:0.040665432810783386 , uid__hotel_roomid_count:0.03696857765316963 , hotel_roomid__hotel_roomid_diff_price_last_lastord_mean:0.03585951775312424 , hotelid__room_30days_ordnumratio_max:0.025508318096399307 , roomtag_1:0.02144177444279194 , hotel_roomid__hotel_roomid_diff_price_last_lastord_max:0.021072087809443474 , uid_user_rank_ratio:0.021072087809443474 , hotel_roomid_room_30days_realratio:0.020702403038740158 , basicroomid__roomtag_1_count:0.019223660230636597 , basicroomid__price_deduct_std:0.015157116577029228 , hotel_roomid__hotel_roomid_diff_basic_minprice_lastord_mean:0.012569315731525421 , basicroomid__room_30days_realratio_mean:0.012569315731525421 , hotelid__basicroomid_count:0.012569315731525421 , uid_user_avgprice_star:0.01219963002949953 , hotelid__price_deduct_min:0.01219963002949953 , roomservice_3:0.01182994432747364 , basicroomid_is_lastord:0.0118299443

In [88]:
pickle.dump(model, open(model_path, 'wb'))